# E1 = Experiment 1 on T5
- Changed Question Length to 150 (from 250)
- Changed Answer Length to 500 (from 32)
- Changed Batch to 4 (from 2)
- Changing Training Approach.

## jump to current running cell 

In [1]:
%%javascript
// Go to Running cell shortcut
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('Alt-I', {
    help : 'Go to Running cell',
    help_index : 'zz',
    handler : function (event) {
        setTimeout(function() {
            // Find running cell and click the first one
            if ($('.running').length > 0) {
                //alert("found running cell");
                $('.running')[0].scrollIntoView();
            }}, 250);
        return false;
    }
});

<IPython.core.display.Javascript object>

# Setting for Kaggle & TPU

In [2]:
# @title Load Configuraiton 
# import hbqaconfig
kaggle=True
# conf = hbqaconfig.setEnv('local')

In [3]:
conf = {'DATA_FOLDER': '/kaggle/input/',
 'PE_FOLDER': '/kaggle/input/',
 'QAGS_FOLDER': '/kaggle/working/',
 'DRS_FOLDER': '/kaggle/working/',
 'AGS_FOLDER': '/kaggle/working/',
 'REPORT_FOLDER': '/kaggle/working/',
 'MODEL_FOLDER': '/kaggle/working/',
 'CORPUS_SECTIONS_FOLDER': '/kaggle/working/',
 'CORPUS_CHAPTER_FOLDER': '/kaggle/working/'}

for k,v in conf.items(): print (k,":",v)

DATA_FOLDER : /kaggle/input/
PE_FOLDER : /kaggle/input/
QAGS_FOLDER : /kaggle/working/
DRS_FOLDER : /kaggle/working/
AGS_FOLDER : /kaggle/working/
REPORT_FOLDER : /kaggle/working/
MODEL_FOLDER : /kaggle/working/
CORPUS_SECTIONS_FOLDER : /kaggle/working/
CORPUS_CHAPTER_FOLDER : /kaggle/working/


In [4]:
# @title Select Processing Device
import torch

# # Detect and initialize TPU
# tpu_available = tf.config.experimental.list_logical_devices("TPU")
# if tpu_available:
#     print("TPU available")
# else:
#     print("No TPU available")

if kaggle:
    DEVICE = torch.device("cuda") #xm.xla_device(fold + 1)
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
    
print(DEVICE)

cuda


In [5]:
predmodels_list = {"0" :"T5:t5", "1" : "Flan-T5:flant5", "2":"GPT2:gpt2","3":"DistilBERT:distilbert",
                   "4":"RoBERTa:roberta","5":"Llama2:llama2", "6":"BERTSquad:bert", 
                   "7":"LongFormer:longformer", "8":"DistilBERT512:distilbert512"}
# Longformer doesn't work with latest Transformer

predict_now=True
embed_now=True
compute_metrics_now=True

# Example Questions

In [6]:
# @title Example Questions
q1 = "What predicament does Yudhishthira face, and how does he seek guidance to resolve it?"

ra1 = "Yudhishthira faces the predicament of being unable to support the Brahmanas who are following him as he departs for the forest. \
He seeks guidance to resolve this dilemma by approaching his priest, Dhaumya, and inquiring about the appropriate course of action."

ra1 = ra1.replace('\n','')

c1 = """Section III
"Vaisampayana said, 'Yudhishthira the son of Kunti, thus addressed by Saunaka, approached his priest and in the midst of his brothers said,
'The Brahmanas versed in the Vedas are following me who am departing for the forest. Afflicted with many calamities I am unable to support them.
I cannot abandon them, nor have I the power to offer them sustenance: Tell me, O holy one, what should be done by me in such a pass.'
"Vaisampayana said, 'After reflecting for a moment seeking to find out the (proper) course by his yoga powers, Dhaumya, that foremost of all virtuous men,
addressed Yudhishthira, in these words, 'In days of old, all living beings that had been created were sorely afflicted with hunger.
And like a father (unto all of them), Savita (the sun) took compassion upon them. And going first into the northern declension,
the sun drew up water by his rays, and coming back to the southern declension, stayed over the earth, with his heat centered in himself.
And while the sun so stayed over the earth, the lord of the vegetable world (the moon), converting the effects of the solar heat (vapours) into clouds
and pouring them down in the shape of water, caused plants to spring up. Thus it is the sun himself, who, drenched by the lunar influence, is transformed,
upon the sprouting of seeds, into holy vegetable furnished with the six tastes. And it is these which constitute the food of all creatures upon the earth.
Thus the food that supporteth the lives of creatures is instinct with solar energy, and the sun is, therefore, the father of all creatures.
Do thou, hence, O Yudhishthira, take refuge even in him. All illustrious monarchs of pure descent and deeds are known to have delivered their people by practising high asceticism.
The great Karttavirya, and Vainya and Nahusha, had all, by virtue of ascetic meditation preceded by vows, delivered their people from heavy afflictions.
Therefore, O virtuous one, as thou art purified by the acts do thou likewise, entering upon a file of austerities. O Bharata, virtuously support the regenerate ones.'
"Janamejaya said, 'How did that bull among the Kurus, king Yudhishthira, for the sake of the Brahmanas adore the sun of wonderful appearance?"
"Vaisampayana said, 'Listen attentively, O king, purifying thyself and withdrawing thy mind from every other thing. And, O king of kings, appoint thou a time.
I will tell thee everything in detail, And, O illustrious one, listen to the one hundred and eight names (of the sun)
as they were disclosed of old by Dhaumya to the high-souled son of Pritha. Dhaumya said, 'Surya, Aryaman, Bhaga, Twastri, Pusha, Arka, Savitri.
Ravi, Gabhastimat, Aja, Kala, Mrityu, Dhatri, Prabhakara, Prithibi, Apa, Teja, Kha, Vayu, the sole stay, Soma, Vrihaspati, Sukra, Budha, Angaraka,
Indra, Vivaswat, Diptanshu, Suchi, Sauri, Sanaichara, Brahma, Vishnu, Rudra, Skanda, Vaisravana, Yama, Vaidyutagni, Jatharagni, Aindhna, Tejasampati,
Dharmadhwaja, Veda-karttri, Vedanga, Vedavahana, Krita, Treta, Dwapara, Kali, full of every impurity, Kala, Kastha, Muhurtta, Kshapa, Yama, and Kshana;
Samvatsara-kara, Aswattha, Kalachakra, Bibhavasu, Purusha, Saswata, Yogin, Vyaktavyakta, Sanatana, Kaladhyaksha, Prajadhyaksha, Viswakarma, Tamounda,
Varuna, Sagara, Ansu, Jimuta, Jivana, Arihan, Bhutasraya, Bhutapati, Srastri, Samvartaka, Vanhi, Sarvadi, Alolupa, Ananta, Kapila, Bhanu, Kamada,
Sarvatomukha, Jaya, Visala, Varada, Manas, Suparna, Bhutadi, Sighraga, Prandharana, Dhanwantari, Dhumaketu, Adideva, Aditisuta, Dwadasatman, Aravindaksha,
Pitri, Matri, Pitamaha, Swarga-dwara, Prajadwara, Mokshadwara, Tripistapa, Dehakarti, Prasantatman, Viswatman, Viswatomukha, Characharatman, Sukhsmatman,
the merciful Maitreya. These are the hundred and eight names of Surya of immeasurable energy, as told by the self-create (Brahma). For the acquisition of prosperity,
I bow down to thee, O Bhaskara, blazing like unto gold or fire, who is worshipped of the gods and the Pitris and the Yakshas, and who is adored by Asuras, Nisacharas,
and Siddhas. He that with fixed attention reciteth this hymn at sunrise, obtaineth wife and offspring and riches and the memory of his former existence,
and by reciting this hymn a person attaineth patience and memory. Let a man concentrating his mind, recite this hymn. By doing so,
he shall be proof against grief and forest-fire and ocean and every object of desire shall be his.'
"Vaisampayana continued, 'Having heard from Dhaumya these words suitable to the occasion, Yudhishthira the just, with heart concentrated within itself and purifying it duly,
became engaged in austere meditation, moved by the desire of supporting the Brahmanas. And worshipping the maker of day with offerings of flowers and other articles,
the king performed his ablutions. And standing in the stream, he turned his face towards the god of day. And touching the water of the Ganges
the virtuous Yudhishthira with senses under complete control and depending upon air alone for his sustenance, stood there with rapt soul engaged in pranayama.
And having purified himself and restrained his speech, he began to sing the hymn of praise (to the sun).'
'Yudhishthira said, "Thou art, O sun, the eye of the universe. Thou art the soul of all corporeal existences. Thou art the origin of all things.
Thou art the embodiment of the acts of all religious men. Thou art the refuge of those versed in the Sankhya philosophy (the mysteries of the 1."""

c1 = c1.replace('\n','')

# Install Necessary Libararies

In [7]:
!pip install -Uq transformers
!pip install -Uq evaluate
!pip install -Uq SentencePiece
!pip install -Uq sentence-transformers
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a46a1a58953e372f71da3547041f9a7da55e57fadec207b18db8a23493bd6726
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [8]:
# !pip install accelerate>=0.20.1
# !pip install transformers[torch]
# # You need to restart the kernel after this step

# Load Libraries and Configurations

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import ast
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F

from sentence_transformers import SentenceTransformer, util

import tensorflow as tf

# import spacy
# import string

from sklearn.model_selection import train_test_split

import transformers
from transformers import pipeline
import evaluate  # Bleu
from transformers import Trainer, TrainingArguments

import nltk
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load QA Dataset

In [10]:
# df = pd.read_csv(datapath + '06-HBQA_Manual_with_Chunk.csv') # for local machine/vedavit colab

# Uncomment below code
# !pip install -q gdown
# import gdown

# # Replace the shared link with the actual link to your file
# file_url = 'https://drive.google.com/uc?id=1Euvnmp8yJ2LGlL2uDvDjER87PYz9RVvS'
# output_path = '/content/hbqa-colab.txt'  # Specify the desired file name and path

# gdown.download(file_url, output_path, quiet=False)

file='06-HBQA_Manual_with_Chunk.csv'
if kaggle: file = file.split(".")[0].lower().replace('_','-') +'/' + file
df = pd.read_csv(conf['PE_FOLDER'] + file)

In [11]:
print(df.shape)
df.head(2)

(1102, 11)


,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk
0,0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809
1,1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852


In [12]:
df.set_index('Ques_Id',inplace=True)
df.head(2)

,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk
Ques_Id,,,,,,,,,,
0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809
1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852


# Common Setting for Training

In [13]:
Question_Len = int(max(len(ques) for ques in df.Question)/4)
Answer_Len = int(max(len(ans) for ans in df.Ref_Answer)/4)
Chunk_Len = int(max(len(chunk) for chunk in df.Chunk)/4)
Chunk_Len, Question_Len,Answer_Len # in Tokens

(1499, 51, 326)

# Load Base Model for Finetuning

In [14]:
class QA_Dataset(Dataset):
  def __init__(self, tokenizer, dataframe, q_len, t_len):
      self.tokenizer = tokenizer
      self.q_len = q_len
      self.t_len = t_len
      self.data = dataframe
      self.questions = self.data["Question"]
      self.context = self.data["Chunk"]
      self.answer = self.data['Ref_Answer']

  def __len__(self):
      return len(self.questions)

  def __getitem__(self, idx):
      question = self.questions[idx]
      context = self.context[idx]
      answer = self.answer[idx]

#       inputs = tokenizer(text, truncation="only_first", max_length=max_length, return_overflowing_tokens=True, return_tensors="pt")
        
      question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                  truncation=True, pad_to_max_length=True, add_special_tokens=True)
      answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                        truncation=True, pad_to_max_length=True, add_special_tokens=True)

      labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
      labels[labels == 0] = -100

      return {
          "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
          "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
          "labels": labels,
          "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
      }

In [15]:
def load_model(predmodel_name):
    if predmodel_name=="flant5":
        from transformers import T5Tokenizer, T5ForConditionalGeneration #,T5Model,  T5TokenizerFast
        model_name = "t5-small" #"t5-base"
        model_name = "google/flan-t5-small"
        tokenizer = T5Tokenizer.from_pretrained(model_name)
        model = T5ForConditionalGeneration.from_pretrained(model_name, return_dict=True)
        model.to(DEVICE)
        optimizer = Adam(model.parameters(), lr=0.00001)
    return tokenizer, model

In [16]:
predmodels_list

{'0': 'T5:t5',
 '1': 'Flan-T5:flant5',
 '2': 'GPT2:gpt2',
 '3': 'DistilBERT:distilbert',
 '4': 'RoBERTa:roberta',
 '5': 'Llama2:llama2',
 '6': 'BERTSquad:bert',
 '7': 'LongFormer:longformer',
 '8': 'DistilBERT512:distilbert512'}

In [17]:
# @title Select prediction model
predmodel_name = predmodels_list['1'].split(':')[1]
print(predmodel_name)
tokenizer, model = load_model(predmodel_name)

flant5


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
# # Earlier Experiment code
# # Dataloader

# train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# train_sampler = RandomSampler(train_data.index)
# val_sampler = RandomSampler(val_data.index)

# qa_dataset = QA_Dataset(tokenizer, df, Q_LEN, T_LEN)

# train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
# val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [19]:
Question_Len = int(max(len(ques) for ques in df.Question)/4)
Answer_Len = int(max(len(ans) for ans in df.Ref_Answer)/4)
Chunk_Len = int(max(len(chunk) for chunk in df.Chunk)/4)
print(Chunk_Len, Question_Len,Answer_Len) # in Tokens

Q_LEN =  1500 #256   # Question Length
T_LEN =  500 #32  # Target Length
BATCH_SIZE = 4
# DEVICE = "cuda:0"


1499 51 326


In [20]:
#E2 Experiment Code
# Dataloader

train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

train_dataset = QA_Dataset(tokenizer, df.loc[train_sampler,:], Q_LEN, T_LEN)
val_dataset = QA_Dataset(tokenizer, df.loc[val_sampler,:], Q_LEN, T_LEN)

# train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
# val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

# Start finetune (Training)

In [ ]:
MODEL_FOLDER = "./flant5/"
# # Set up the training arguments
training_args = TrainingArguments(
    output_dir=MODEL_FOLDER,
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=30,
    save_steps=1000, # Set the frequency to save the model
    eval_steps=1000,  # Set the frequency to evaluate the model
    save_total_limit=1, # Limit the number of saved checkpoints to 1 (best model)
    logging_dir='./logs', # learning_rate=2e-5,
    evaluation_strategy="epoch",  # Change this to "steps"/ "epoch" if you want to evaluate at the end of each epoch
    remove_unused_columns=False,
    push_to_hub=False,  # Set to True if you want to push the model to the Hugging Face Hub
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,

)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,1.754216
2,No log,1.651704
3,1.983400,1.571578
4,1.983400,1.500634
5,1.748900,1.451583
6,1.748900,1.396054
7,1.633100,1.352686
8,1.633100,1.311440
9,1.633100,1.281806
10,1.546400,1.250596


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

# Save/ Load Finetuned (trained) Model

In [ ]:
model.save_pretrained(MODEL_FOLDER)
tokenizer.save_pretrained(MODEL_FOLDER)

# Saved files
"""('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
'qa_tokenizer/added_tokens.json',
'qa_tokenizer/tokenizer.json')"""

## Load Model from Memory

In [ ]:
conf['MODEL_FOLDER'] = '/kaggle/working/flant5'

In [ ]:
# from transformers import T5ForConditionalGeneration, T5Tokenizer

# # Load the corresponding tokenizer
# tokenizer = T5Tokenizer.from_pretrained( conf['MODEL_FOLDER'])

# # Load the pre-trained T5 model
# model = T5ForConditionalGeneration.from_pretrained(MODEL_FOLDER)
# model.to(DEVICE)

In [ ]:
# input_text = "Translate the following English text to French: 'Hello, how are you?'"

# # Tokenize the input text
# input_ids = tokenizer.encode(input_text, return_tensors='pt')
# input_ids = input_ids.to(DEVICE)

# # Generate translated text
# translated_ids = MODEL.generate(input_ids)

# # Decode the generated IDs back to text
# translated_text = TOKENIZER.decode(translated_ids[0], skip_special_tokens=True)

# print("Translated Text:", translated_text)


# Predict Answers from t5 model

In [ ]:
# @title Predict Answer Function
def predict_answer(context, question, tokenizer, model):
#     Q_LEN=1500
    
#     if predmodel_name=="t5":
    inputs = tokenizer(question, context, max_length= Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100)

    predicted_answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)
 

    if len(predicted_answer)<3:
       predicted_answer="xxx" 
    elif predicted_answer[0:5]=='[CLS]' or  predicted_answer[0:5]=='[SEP]' or predicted_answer[0:3]=='<s>' :
        predicted_answer="xxx"
    return predicted_answer

In [ ]:
# @title Predict Answer Function
from IPython.display import clear_output
clear_output()

from IPython.display import display
from IPython.display import HTML

if predict_now:
  df_pred = pd.DataFrame(columns=['Ques_Id','Question','Ref_Answer','Pred_Answer'])

def predict_all_answers(df_pred, sample=True, verbose=False):
  import random
  if sample:
    qno=random.sample(set(df.index),10)
  else:
    qno = df.index

  total_items = len(qno)
  j=0

  # progress_bar = tqdm(total=total_items, position=0, leave=True)

  progress_bar = tqdm(total=total_items, position=0, leave=True, dynamic_ncols=True)
  progress_html = HTML("""<progress value="0" max="{0}" style="width: 100%"></progress>""".format(total_items))


  for i  in qno:
      progress_bar.update(j)
      progress_html.value = """<progress value="{0}" max="{1}" style="width: 100%"></progress>""".format(i, total_items)
      progress_bar = tqdm(total=total_items, position=0, leave=True)
      ques_id  = df.index[i]
      chunk    = df.iloc[i]['Chunk']
      # print(chunk)
      # chunk = chunk.replace("'","").replace("\","")
      ques     = df.iloc[i]['Question']
      ref_ans  = df.iloc[i]['Ref_Answer']

      pred_ans = predict_answer(chunk, ques, tokenizer, model)

      df_pred.loc[j] = ques_id, ques, ref_ans, pred_ans
      j+=1
      if verbose:
        print('Question  :', ques)
        print("Ref Answer:", ref_ans)
        print("Pred Ans  :", pred_ans)
        print('--------')

      # print(f"Prediting Answer {i}/{df.shape[0]}")

  # progress_bar.close()

In [28]:
# @title Start Predicting Answer
predicted_ans_file = '09.21-' + predmodel_name +'Predicted_Ans-Finetuned.csv'
if predict_now:
  predict_all_answers(df_pred, sample=False, verbose=False)

if predict_now:
  print('Saving Predictions')
  df_pred.to_csv(conf['AGS_FOLDER'] + predicted_ans_file, index=False)

  0%|          | 3/1102 [00:01<07:09,  2.56it/s]
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  0%|          | 4/1102 [00:00<03:59,  4.59it/s]
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  0%|          | 5/1102 [00:01<04:16,  4.28it/s]
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  1%|          | 6/1102 [00:00<02:55,  6.23it/s]
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' tr

Saving Predictions


# Load Embedding Model

In [29]:
#Select Model Function

# https://www.sbert.net/docs/pretrained_models.html

#250MB, multi-qa-distilbert-cos-v1',  Max Sequence Length:	512, Dimensions:768, Normalized Embeddings:	true
#80MB, all-MiniLM-L6-v2, Max Sequence Length:	256, Dimensions:	384, Normalized Embeddings:	true
#290MB, all-distilroberta-v1, Max Sequence Length:	512, Dimensions:	768, Normalized Embeddings:	true
#420MB, all-mpnet-base-v2, Max Sequence Length:	384, Dimensions:	768, Normalized Embeddings:	true
#1.36GB, all-roberta-large-v1, Max Sequence Length:	256, Dimensions: 1024, Normalized Embeddings:	true

def select_embmodel(num):
    emb_modelshortlist = ['distilbert','minilm','distilroberta','mpnet','roberta']

    emb_modellist = ['multi-qa-distilbert-cos-v1',
                'all-MiniLM-L6-v2',
                'all-distilroberta-v1',
                'multi-qa-mpnet-base-dot-v1',
                'all-roberta-large-v1']

    embmodelname = emb_modellist[num]
    embmodelshort = emb_modelshortlist[num]
    embmodelname1 = "_" + embmodelname

    print (embmodelname,'\t',embmodelshort,'\t', embmodelname1)
    return embmodelname, embmodelshort, embmodelname1

In [30]:
embmodelname, embmodelshort, embmodelname1 = select_embmodel(3)

multi-qa-mpnet-base-dot-v1 	 mpnet 	 _multi-qa-mpnet-base-dot-v1


In [31]:
embed_now=True

In [32]:
if embed_now:
  embmodel = SentenceTransformer(embmodelname)

In [33]:
predicted_ans_file = '09.21-' + predmodel_name +'Predicted_Ans-Finetuned.csv'
df_pred =  pd.read_csv(conf['AGS_FOLDER'] + predicted_ans_file)

df_pred.set_index('Ques_Id',inplace=True)

In [34]:
df_pred

,Question,Ref_Answer,Pred_Answer
Ques_Id,,,
0,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,The Agnihotra is considered a significant mora...
1,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,Yudhishthira faces a predicament of being unab...
2,What advice does Dhaumya offer to Yudhishthira...,Dhaumya advises Yudhishthira to take refuge in...,Dhaumya advises Yudhishthira to purify himself...
3,"How did Yudhishthira adore the sun, and what i...",Yudhishthira adored the sun by performing ablu...,Yudhishthira adored the sun by purifying himse...
4,What are the hundred and eight names of the su...,"The hundred and eight names of the sun, as rev...",The hundred and eight names of the sun as reve...
...,...,...,...
1097,"In this passage, how does Dhritarashtra descri...",Dhritarashtra describes Lord Krishna (Vasudeva...,Dhritarashtra describes Lord Krishna as a slay...
1098,What does Yudhishthira request of Krishna in t...,Yudhishthira requests Krishna to go to the Kur...,Yudhishthira asks Krishna to go to the Kurus t...
1099,What is Sahadeva's desire and the reason behin...,Sahadeva desires to provoke a war with the Kau...,Sahadeva is a virtuous and virtuous warrior wh...


In [35]:
def create_ans_vector():

    Ans_Sentences = df_pred.Pred_Answer.tolist()
    Ans_Embeddings = embmodel.encode(Ans_Sentences)

    print ("Answer Embedding: ", Ans_Embeddings.shape)
    # return (Ques_Embeddings, Ans_Embeddings)

    return (Ans_Embeddings)

In [36]:
# @title Answer not found these questions
df_pred['Pred_Answer'] = df_pred.Pred_Answer.fillna("xxx")
df_pred.loc[df_pred['Pred_Answer'].str.contains(r'\?\?\?'), 'Pred_Answer'] = "xxx"

# Model couldn't fine any any answer for this
df_pred.loc[df_pred['Pred_Answer'].str.contains(r"xxx")].shape

(0, 3)

In [37]:
if embed_now:
  Ans_Embeddings = create_ans_vector()
  df_pred['Pred_Answer_Vector'] = torch.tensor(Ans_Embeddings, dtype=torch.float).tolist()

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Answer Embedding:  (1102, 768)


In [38]:

predicted_ansvec_file = '09.21-' + predmodel_name +'Predicted_AnsVec-Finetuned.csv'
if embed_now:
  df_pred.to_csv(conf['AGS_FOLDER'] + predicted_ansvec_file)

# df_pred = pd.read_csv(datapath + '09.11-' + predmodel_name +'Predicted_AnsVec-Zeroshot.csv')
print(df_pred.shape)
df_pred.head(2)

(1102, 4)


,Question,Ref_Answer,Pred_Answer,Pred_Answer_Vector
Ques_Id,,,,
0,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,The Agnihotra is considered a significant mora...,"[-0.38374489545822144, -0.11841078847646713, -..."
1,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,Yudhishthira faces a predicament of being unab...,"[-0.04933479428291321, -0.2885929346084595, -0..."


# Calculate Metrics

In [39]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics import precision_score, recall_score
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
import numpy as np

smoother = SmoothingFunction()
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def get_nlp_metrics(ques_id, ref_ans, pred_ans):

  # Calculate ROUGE-1 and ROUGE-L scores
  scores = scorer.score(ref_ans, pred_ans)

  # Access individual ROUGE scores
  rouge1_precision = scores['rouge1'].precision
  rouge1_recall = scores['rouge1'].recall
  rouge1_f1 = scores['rouge1'].fmeasure

  rougel_precision = scores['rougeL'].precision
  rougel_recall = scores['rougeL'].recall
  rougel_f1 = scores['rougeL'].fmeasure

#   rouge_1_precision, rouge_1_recall, rouge_1_f1, rouge_l_precision, rouge_l_recall, rouge_l_f1,

  ref_tokens = word_tokenize(ref_ans)
  pred_tokens = word_tokenize(pred_ans)

  # Calculate BLEU score for a single sentence
  bleu_score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoother.method2)

  # Calculate BLEU score for multiple sentences
  # corpus_bleu_score = corpus_bleu([[ref_tokens]], [pred_tokens],  smoothing_function=smoother.method2)

  # print(set(ref_tokens).intersection(set(pred_tokens)))
  tp = len(set(ref_tokens).intersection(set(pred_tokens)))
  precision = tp / len(pred_tokens)
  recall =  tp / len(ref_tokens)

  return ques_id, bleu_score, rouge1_precision, rouge1_recall, rouge1_f1, rougel_precision, rougel_recall, rougel_f1, precision, recall

In [40]:
compute_metrics_now=True

In [41]:
# from datasets import load_metric
if compute_metrics_now:
  df_metrics = pd.DataFrame(columns = ['Ques_Id','BLEU1', 'ROUGE1_P', 'ROUGE1_R', 'ROUGE1_F1',
                              'ROUGEL_P', 'ROUGEL_R', 'ROUGEL_F1', 'Precision', 'Recall'])
  N= df_pred.shape[0]

  for i in range(N):
    # bleu_score1 = calculate_score( df1.iloc[i]['ref_answer'],df1.iloc[i]['pred_answer'])
    ques_id = df_pred.index[i]
    ref_ans  = df_pred.loc[df_pred.index==ques_id,'Ref_Answer'].values[0]
    pred_ans = df_pred.loc[df_pred.index==ques_id,'Pred_Answer'].values[0]
    results = get_nlp_metrics(ques_id, ref_ans, pred_ans)
    df_metrics.loc[i] = results

In [42]:
# !rm /kaggle/working/09.21-t5Predicted_AnsVec-Zeroshot.csv

In [43]:
metricsfile = '09.22-' + predmodel_name +'Predicted_Ans_Score_Finetuned.csv'
print(metricsfile)

if compute_metrics_now:
  df_metrics.to_csv(conf['REPORT_FOLDER'] + metricsfile, index=False)

# df_metrics = pd.read_csv(datapath + metricsfile)
print(df_metrics.shape)
df_metrics.head(3)

09.22-flant5Predicted_Ans_Score_Finetuned.csv
(1102, 10)


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGEL_F1,Precision,Recall
0,0.0,0.104145,0.232143,0.260000,0.245283,0.196429,0.220000,0.207547,0.180328,0.180328
1,1.0,0.268711,0.430769,0.682927,0.528302,0.384615,0.609756,0.471698,0.333333,0.533333
2,2.0,0.126796,0.372881,0.431373,0.400000,0.271186,0.313725,0.290909,0.296875,0.316667


In [44]:
# df_metrics = pd.read_csv(conf['REPORT_FOLDER'] + metricsfile)
print(df_metrics.shape)
df_metrics.head(3)

(1102, 10)


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGEL_F1,Precision,Recall
0,0.0,0.104145,0.232143,0.260000,0.245283,0.196429,0.220000,0.207547,0.180328,0.180328
1,1.0,0.268711,0.430769,0.682927,0.528302,0.384615,0.609756,0.471698,0.333333,0.533333
2,2.0,0.126796,0.372881,0.431373,0.400000,0.271186,0.313725,0.290909,0.296875,0.316667


# Calculate Cosine between Predicted and Reference Answer

In [45]:
# filenm = r'07.2-HBQA_Cosine_multi-qa-mpnet-base-dot-v1.csv'
filenm = '07.2-HBQA_QA_Vector' + embmodelname1 + '.csv'
filenm = '07-2-hbqa-cosine-multi-qa-mpnet-base-dot-v1/07.2-HBQA_Cosine_multi-qa-mpnet-base-dot-v1.csv'
# if kaggle: filenm = filenm.split(".")[0].lower().replace('_','-') +'/' + filenm
    
df_ref_ans = pd.read_csv(conf['PE_FOLDER'] + filenm)

df_ref_pred= df_pred.merge(df_ref_ans, on = "Ques_Id", how="inner")[['Ques_Id','Pred_Answer_Vector','AnsVector_multi-qa-mpnet-base-dot-v1']]

In [46]:
def calculate_cosine(row):

    if type(row['Pred_Answer_Vector'])!=type(list()):
      predAns_vector_values = ast.literal_eval(row['Pred_Answer_Vector'])
    else:
      predAns_vector_values = row['Pred_Answer_Vector']

    if type(row['AnsVector' + embmodelname1])!=type(list()):
      refAns_vector_values = ast.literal_eval(row['AnsVector' + embmodelname1])
    else:
      refAns_vector_values = row['AnsVector' + embmodelname1]


    # Convert the string values to floats
    predAns_vector_values = [float(value) for value in predAns_vector_values]
    refAns_vector_values = [float(value) for value in refAns_vector_values]

    # Convert the lists to PyTorch tensors
    predAns_vector_values = torch.tensor(predAns_vector_values).reshape(1, -1)
    refAns_vector_values = torch.tensor(refAns_vector_values).reshape(1, -1)

    similarity = cosine_similarity(predAns_vector_values, refAns_vector_values)
    return similarity.item()

In [47]:
df_ref_pred['Cosine'] = df_ref_pred.apply(calculate_cosine, axis=1)

In [48]:
df_metrics = df_metrics.merge(df_ref_pred, on="Ques_Id", how="left")[['Ques_Id', 'BLEU1', 
                                                                      'ROUGE1_P', 'ROUGE1_R', 'ROUGE1_F1', 
                                                                      'ROUGEL_P', 'ROUGEL_R', 'ROUGEL_F1', 
                                                                      'Precision', 'Recall','Cosine']]

# Save File Metrics & Show Summary

In [49]:
metricsfile = '09.22-' + predmodel_name +'Predicted_Ans_Score_Finetuned.csv'
print(metricsfile)
df_metrics.to_csv(conf['REPORT_FOLDER']+ metricsfile, index=False)
df_metrics.head(2)

09.22-flant5Predicted_Ans_Score_Finetuned.csv


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGEL_F1,Precision,Recall,Cosine
0,0.0,0.104145,0.232143,0.260000,0.245283,0.196429,0.220000,0.207547,0.180328,0.180328,0.687688
1,1.0,0.268711,0.430769,0.682927,0.528302,0.384615,0.609756,0.471698,0.333333,0.533333,0.765529


In [50]:
all_metrics = pd.DataFrame()
all_metrics[predmodel_name] = df_metrics.mean()[1:]

In [51]:
# all_metrics.rename(columns = {'t5':'t5-small-1epoch'}, inplace=True)
all_metrics

,flant5
BLEU1,0.204073
ROUGE1_P,0.483073
ROUGE1_R,0.477008
ROUGE1_F1,0.465062
ROUGEL_P,0.386867
ROUGEL_R,0.386015
ROUGEL_F1,0.374271
Precision,0.356957
Recall,0.356533
Cosine,0.825831


In [52]:
df_metrics.mean()

Ques_Id      550.500000
BLEU1          0.204073
ROUGE1_P       0.483073
ROUGE1_R       0.477008
ROUGE1_F1      0.465062
ROUGEL_P       0.386867
ROUGEL_R       0.386015
ROUGEL_F1      0.374271
Precision      0.356957
Recall         0.356533
Cosine         0.825831
dtype: float64

# Download files from Kaggle

In [53]:
!zip -r file.zip /kaggle/working

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/logs/ (stored 0%)
  adding: kaggle/working/logs/events.out.tfevents.1697825629.da957634b62f.32.0 (deflated 65%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/wandb/ (stored 0%)
  adding: kaggle/working/wandb/debug-internal.log (deflated 95%)
  adding: kaggle/working/wandb/latest-run/ (stored 0%)
  adding: kaggle/working/wandb/latest-run/logs/ (stored 0%)
  adding: kaggle/working/wandb/latest-run/logs/debug-internal.log (deflated 95%)
  adding: kaggle/working/wandb/latest-run/logs/debug.log (deflated 81%)
  adding: kaggle/working/wandb/latest-run/tmp/ (stored 0%)
  adding: kaggle/working/wandb/latest-run/tmp/code/ (stored 0%)
  adding: kaggle/working/wandb/latest-run/run-y4flqtdo.wandb (deflated 90%)
  adding: kaggle/working/wandb/latest-run/files/ (stored 0%)
  adding: kaggle/working/wandb/latest-run/files/requirements.txt (deflated 58%)
  adding: kaggle/working/wandb/latest-run/files/wan

In [54]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip